# Importación de librerias


In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import requests
import gdown
import zipfile
import matplotlib.pyplot as plt
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
url = '1mMaUvQuLaEOcwekPww2RGsfZjxCaqT_q'
gdown.download(f"https://drive.google.com/uc?export=download&id={url}", 'kaggle.json', quiet=False)
os.makedirs("/root/.kaggle", exist_ok=True)
shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia
zip_file_path = '/content/udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip'

if os.path.exists(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall()

In [ ]:
df = pd.read_csv('/content/train.csv')

Se eligen las variables y la variable objetivo

In [ ]:
y = df['RENDIMIENTO_GLOBAL']
X = df.drop('RENDIMIENTO_GLOBAL', axis=1)

In [22]:
def hallar_columnasNaN(df):
  k = df.isna().sum()
  return k[k!=0]

# Metodo One-Hot

In [23]:
from sklearn.preprocessing import OneHotEncoder

def one_hot(df, column):
  codificador = OneHotEncoder()
  codificacion = codificador.fit_transform(df[[column]])
  categorias = list(f"{column}_{str(i)}" for i in codificador.categories_[0])
  nuevas_columnas = pd.DataFrame(codificacion.toarray(), columns=categorias)
  df = pd.concat([df, nuevas_columnas], axis=1)
  df = df.drop(column, axis=1)
  return df

# LabelEncoding

In [24]:
from sklearn.preprocessing import LabelEncoder

def label_encode(df, columns):
  for column in columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
  return df

# Imputación por moda

In [25]:
def imputacion_por_moda(df):
  for column in df.columns:
    df_aux = df.copy()
    df_aux[column] = df_aux[column].fillna(df_aux[column].mode().iloc[0])
  return df_aux

# Imputación KNN


In [26]:
from sklearn.impute import KNNImputer

def imputacion_KNN(df):
  columns_NaN = hallar_columnasNaN(df)
  for i in df.columns:
    if i in columns_NaN:
      df = one_hot(df, i)
    else:
      df = label_encode(df, [i])
  imputador = KNNImputer(n_neighbors=3, weights='uniform')
  df_imputado = pd.DataFrame(
    imputador.fit_transform(df),
    columns=df.columns
  )
  return df_imputado

# Evaluar modelo

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

def evaluar_modelo(nombre, modelo, X_test, y_test):
    y_pred = modelo.predict(X_test)

    print(f"--- Evaluación para {nombre} ---")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred, average='weighted'))
    print("Recall:", recall_score(y_test, y_pred, average='weighted'))
    print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))
    print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))
    print()

In [ ]:
imputacion_KNN(X)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_PRIVADO_LIBERTAD,coef_1,coef_2,coef_3,coef_4,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 1 millón y menos de 2.5 millones,...,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE_nan
0,632903.0,7.0,300.0,4.0,0.0,212.0,143.0,185.0,210.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,451863.0,7.0,249.0,3.0,0.0,201.0,150.0,167.0,207.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,215840.0,6.0,819.0,4.0,0.0,187.0,149.0,180.0,207.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,329259.0,3.0,14.0,26.0,0.0,270.0,121.0,127.0,133.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,692332.0,7.0,907.0,1.0,0.0,206.0,167.0,160.0,237.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,17648.0,3.0,144.0,17.0,0.0,160.0,189.0,146.0,254.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
692496,528108.0,7.0,907.0,21.0,0.0,204.0,175.0,153.0,203.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
692497,353010.0,0.0,94.0,4.0,0.0,176.0,175.0,189.0,230.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
692498,690637.0,3.0,907.0,28.0,0.0,55.0,288.0,136.0,271.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
X_new = imputacion_KNN(X)

# Modelo de clasificación lineal

In [29]:
from sklearn.linear_model import LogisticRegression

# Crear el modelo
modelo = LogisticRegression(max_iter=1000, solver='saga', n_jobs=-1)

# Entrenar
modelo.fit(X_new, y)

# Predecir
y_pred = modelo.predict(X_new)
print(y_pred)

KeyboardInterrupt: 

# Árboles de decisión

In [31]:
from sklearn.tree import DecisionTreeClassifier

modelo = DecisionTreeClassifier(max_depth=5)  # max_depth es opcional
modelo.fit(X_new, y)
y_pred = modelo.predict(X_new)

In [33]:
evaluar_modelo('Decision Tree', modelo, X_new, y)

--- Evaluación para Decision Tree ---
Accuracy: 0.36947003610108303
Precision: 0.3498365344521748
Recall: 0.36947003610108303
F1 Score: 0.34549829493108364
Matriz de Confusión:
 [[99381 28608 34114 13516]
 [28496 93866 31567 19058]
 [56631 53781 39846 21361]
 [39357 74199 35954 22765]]



# Naives Bayes

In [34]:
from sklearn.naive_bayes import GaussianNB

modelo = GaussianNB()
modelo.fit(X_new, y)
y_pred = modelo.predict(X_new)

In [35]:
evaluar_modelo('Naives Bayes', modelo, X_new, y)

--- Evaluación para Naives Bayes ---
Accuracy: 0.32538483754512637
Precision: 0.3313821763988819
Recall: 0.32538483754512637
F1 Score: 0.3230025058773222
Matriz de Confusión:
 [[72442 16193 38610 48374]
 [30758 46661 29653 65915]
 [44497 25892 38616 62614]
 [34833 34242 35590 67610]]



# Support Vector Machine (SVM)

In [ ]:
from sklearn.svm import SVC

# Modelo SVM con kernel lineal
modelo = SVC(kernel='linear', probability=True)

# O kernel RBF (por defecto)
# modelo = SVC(kernel='rbf', probability=True)

modelo.fit(X_new, y)
y_pred = modelo.predict(X_new)

In [ ]:
evaluar_modelo('SVM', modelo, X_new, y)